In [2]:
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import torch.optim as optim

In [3]:


# Beispiel für Forex-Daten (EUR/USD in 15-Minuten-Intervallen)
# Daten sollten als numpy-Array vorliegen: (Anzahl der Beobachtungen, 1)
forex_data = np.random.rand(1000, 1)  # Dummy-Daten, ersetzen Sie dies durch Ihre Forex-Daten

# Skalierung der Daten
scaler = MinMaxScaler()
forex_data = scaler.fit_transform(forex_data)

# Parameter
sequence_length = 96  # 1 Tag in 15-Minuten-Intervallen
forecast_horizon = 1  # Vorhersage von 1 Zeitschritt (15 Minuten) in die Zukunft

# Funktion zur Erstellung von Eingabesequenzen und Zielwerten
def create_sequences(data, seq_length, horizon):
    inputs = []
    targets = []
    for i in range(len(data) - seq_length - horizon + 1):
        input_seq = data[i:i + seq_length]
        target_seq = data[i + seq_length:i + seq_length + horizon]
        inputs.append(input_seq)
        targets.append(target_seq)
    return np.array(inputs), np.array(targets)

# Erstellen der Eingabesequenzen und Zielwerte
X, y = create_sequences(forex_data, sequence_length, forecast_horizon)

# Umwandlung in PyTorch-Tensoren
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)


In [4]:

# Aufteilung in Trainings-, Validierungs- und Testdatensätze
train_size = int(0.7 * len(X))
val_size = int(0.15 * len(X))
test_size = len(X) - train_size - val_size

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size + val_size], y[train_size:train_size + val_size]
X_test, y_test = X[train_size + val_size:], y[train_size + val_size:]

# Dataset und DataLoader für PyTorch
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Train size: {len(train_loader.dataset)}")
print(f"Validation size: {len(val_loader.dataset)}")
print(f"Test size: {len(test_loader.dataset)}")


Train size: 632
Validation size: 135
Test size: 137


In [5]:


class VARModule(nn.Module):
    def __init__(self, input_size, k, d):
        super(VARModule, self).__init__()
        self.k = k
        self.d = d
        self.linear_transform = nn.Linear(input_size * k, input_size * k)
        self.nonlinear_transform = nn.Linear(input_size * (d * (d + 1)) // 2, input_size * d)

    def forward(self, x):
        batch_size, seq_len, input_size = x.size()
        linear_features = []
        nonlinear_features = []

        for t in range(self.k, seq_len):
            linear_feature = x[:, t-self.k:t, :].reshape(batch_size, -1)
            linear_features.append(self.linear_transform(linear_feature))

            nonlinear_feature = torch.cat([torch.mul(x[:, t-i, :], x[:, t-j, :])
                                           for i in range(1, self.d + 1)
                                           for j in range(i, self.d + 1)], dim=-1)
            nonlinear_features.append(self.nonlinear_transform(nonlinear_feature))

        linear_features = torch.stack(linear_features, dim=1)
        nonlinear_features = torch.stack(nonlinear_features, dim=1)
        return linear_features, nonlinear_features

class BiLSTMModule(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.2):
        super(BiLSTMModule, self).__init__()
        self.bilstm = nn.LSTM(input_size, hidden_size, num_layers, 
                              batch_first=True, bidirectional=True, dropout=dropout)
        self.hidden_size = hidden_size

    def forward(self, x, h_c=None):
        out, (h, c) = self.bilstm(x, h_c)
        return out, (h, c)

class DAFA_BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, latent_space_dim, num_layers, k, d):
        super(DAFA_BiLSTM, self).__init__()
        self.var_module = VARModule(input_size, k, d)
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.bilstm_layers = nn.ModuleList([BiLSTMModule(input_size + input_size * k, hidden_size, 1) for _ in range(num_layers)])
        self.fc = nn.Linear(hidden_size * 2, input_size)  # Bidirectional LSTM doubles the hidden size

    def forward(self, x):
        linear_features, nonlinear_features = self.var_module(x)
        out = nonlinear_features
        for i in range(self.num_layers):
            out, _ = self.bilstm_layers[i](torch.cat([out, linear_features], dim=-1))
        out = self.fc(out)
        return out

def loss_function(recon_x, x):
    return nn.MSELoss()(recon_x, x)



In [6]:

# Hyperparameter
input_size = 1  # Dimension der Forex-Daten
hidden_size = 64  # Dimension des versteckten Zustands
latent_space_dim = 32  # Dimension des latenten Raums
num_layers = 3  # Anzahl der BiLSTM-Schichten
k = 3  # Zeitverzögerungslänge
d = 2  # Ordnung der nichtlinearen Merkmale
sequence_length = 96  # Länge der Eingabesequenz (1 Tag bei 15-Minuten-Abständen)
batch_size = 16
num_epochs = 1
learning_rate = 1e-3


In [7]:

# Initialisierung des Modells, Optimierers und der Loss-Funktion
model = DAFA_BiLSTM(input_size, hidden_size, latent_space_dim, num_layers, k, d)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



C:\Users\nilsw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import subprocess

# Funktion zur Überwachung des GPU-Speicherverbrauchs
def get_gpu_memory_usage():
    try:
        result = subprocess.check_output(['nvidia-smi', '--query-gpu=memory.used', '--format=csv,noheader,nounits'])
        gpu_memory = int(result.decode().strip().split('\n')[0])
        return gpu_memory
    except Exception as e:
        print(f"Error: {e}")
        return 0

# Definition der Verlustfunktion
def loss_function(recon_x, x):
    return nn.MSELoss()(recon_x, x)

# Überprüfen, ob eine GPU verfügbar ist
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialisierung des Modells, Optimierers und der Verlustfunktion
model = DAFA_BiLSTM(input_size, hidden_size, latent_space_dim, num_layers, k, d).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Hyperparameter
accumulation_steps = 4  # Anzahl der Mini-Batches zur Gradientenakkumulation

# Trainingsschleife
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    accumulated_loss = 0
    optimizer.zero_grad()  # Verschieben des Zero-Grad-Aufrufs vor die Schleife
    for i, (batch_x, batch_y) in enumerate(train_loader):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)  # Verschieben der Tensoren auf die GPU
        recon_x = model(batch_x)
        loss = loss_function(recon_x, batch_y)
        loss = loss / accumulation_steps  # Verlust für die Akkumulation normalisieren
        loss.backward()
        
        accumulated_loss += loss.item()
        
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
            optimizer.step()
            optimizer.zero_grad()
            train_loss += accumulated_loss
            accumulated_loss = 0

            # Ausgabe des GPU-Speicherverbrauchs nach jeder Akkumulation
            gpu_memory_usage = get_gpu_memory_usage()
            print(f"Epoch {epoch + 1}, Batch {i + 1}, GPU Memory Usage: {gpu_memory_usage} MB")
    
    val_loss = 0
    model.eval()
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)  # Verschieben der Tensoren auf die GPU
            recon_x = model(batch_x)
            loss = loss_function(recon_x, batch_y)
            val_loss += loss.item()

    print(f'Epoch {epoch + 1}, Training Loss: {train_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}')

print("Training abgeschlossen")


Using device: cpu


C:\Users\nilsw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
C:\Users\nilsw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 93, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


: 

In [ ]:

# Vorhersage mit dem trainierten Modell
model.eval()
with torch.no_grad():
    test_data = X_test  # Verwenden Sie Ihre Testdaten
    predictions = model(test_data)
    print("Predictions shape:", predictions.shape)


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Beispiel-Daten
# Annahme: Eingabezeitreihe mit 10 Features und Ziel-Zeitreihe mit 1 Zielwert pro Zeitpunkt
input_sequence = torch.tensor(np.random.randn(100, 10), dtype=torch.float32).unsqueeze(0)  # 100 Zeitpunkte mit jeweils 10 Features
target_sequence = torch.tensor(np.random.randn(100, 1), dtype=torch.float32).unsqueeze(0) # Ziel-Zeitreihe mit 1 Zielwert pro Zeitpunkt

# Modell definieren
class StackedBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(StackedBiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)  # output_size = 1 für unsere Ziel-Zeitreihe

    def forward(self, x):
        # Initialisierung der Hidden und Cell States für die LSTM-Schichten
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)  # 2 für bidirektional
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)
        
        # LSTM-Schichten
        out, _ = self.lstm(x, (h0, c0))  # out shape: (batch_size, seq_length, hidden_size * 2)
        
        # Nur den letzten Zeitpunkt betrachten
        out = out[:, -1, :]  # Shape jetzt (batch_size, hidden_size * 2)
        
        # Dense-Schicht für die Ausgabe
        out = self.fc(out)  # Shape jetzt (batch_size, output_size)
        return out

# Hyperparameter
input_size = 10
hidden_size = 64
num_layers = 3
output_size = 1

# Modellinstanziierung
model = StackedBiLSTM(input_size, hidden_size, num_layers, output_size)

# Loss-Funktion und Optimierer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training des Modells
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(input_sequence)
    loss = criterion(outputs, target_sequence)
    loss.backward()
    optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Modellvorhersagen
model.eval()
with torch.no_grad():
    predictions = model(input_sequence)
    # Beispiel für den Vergleich der Vorhersagen mit der Ziel-Zeitreihe (hier nur Ausgabe der ersten Vorhersage)
    print("Erste Vorhersage:", predictions[0].item())
    print("Erster tatsächlicher Wert:", target_sequence[0, 0].item()) 


Epoch [1/10], Loss: 0.9003
Epoch [2/10], Loss: 0.8994
Epoch [3/10], Loss: 0.8999
Epoch [4/10], Loss: 0.8997
Epoch [5/10], Loss: 0.8994
Epoch [6/10], Loss: 0.8994
Epoch [7/10], Loss: 0.8995
Epoch [8/10], Loss: 0.8996
Epoch [9/10], Loss: 0.8995
Epoch [10/10], Loss: 0.8994
Erste Vorhersage: 0.02563880756497383
Erster tatsächlicher Wert: 1.4215596914291382
